This notebook contained some exploratory analysis and some filtering that was applied to the csv files used as Documents of our RAG.

In [1]:
import pandas as pd
import numpy as np

# Recipes

## Data set 

* [https://www.kaggle.com/datasets/wilmerarltstrmberg/recipe-dataset-over-2m](https://www.kaggle.com/datasets/wilmerarltstrmberg/recipe-dataset-over-2m)

## Tasks performed:

* Reduce the number of recipecies to a managable number.
* filter non-essential columns,
* Formulate a sentence following the format: 
```
f'To cook {recipe_title}, you need: {ingredients}.'
```

In [ ]:
# read specific columns of csv file using Pandas
df = pd.read_csv("Data/Recipes/recipes_data.csv", usecols=['title', 'ingredients'])
df.loc[:,'Sentence'] = ('To cook ' + df.loc[:,'title'] 
                        + ', you need:' + df.loc[:,'ingredients'].astype(str).str.replace('[','').str.replace(']','').str.replace('"','').str.replace('["',''))

df.rename(columns={'title':'Recipe Title'}, inplace=True)
df = df[['Recipe Title','Sentence']]
df = df.sample(frac=1)
df.reset_index(drop=True, inplace=True)
df.iloc[0::10].to_csv(f'recipes_1.csv',index=False)

# Agriculture Production by country

## Data set:

* https://www.fao.org/faostat/en/#data/QCL

## Tasks performed:

* Filter to keep only data from the last surveys,
* Remove values expressed in millions when the same valmue is available as a percentage,
* filter non-essential columns,
* Formulate a sentence based on the remaining columns: 
```
f'The production of {Item} in {Area} is {Value} {Unit} per year'
```

In [ ]:
df_prod =pd.read_csv("Data/Production/Production_Crops_Livestock_E_All_Data_(Normalized).csv")
print(df_prod.shape)

def group_function(x):
    return pd.Series({'Max Year Code': np.max(x['Year Code'])})

group_features = ['Area Code', 'Item Code','Element Code']

grouped_df = df_prod[group_features+['Year Code']].groupby(by = group_features).apply(group_function).reset_index()
df_prod = df_prod.merge(grouped_df, on = group_features, how= 'outer')
df_prod = df_prod[df_prod['Year Code']==df_prod['Max Year Code']]
df_prod = df_prod[df_prod['Element'].isin(['Production'])]
columns_to_keep = ['Area','Item', 'Unit','Value']
df_prod = df_prod[columns_to_keep]
df_prod.loc[:,'Sentence'] = ('The production of '
                             +df_prod.loc[:,'Item']+' in '+df_prod.loc[:,'Area']
                             +' is '+df_prod.loc[:,'Value'].astype(str)
                             +' '+df_prod.loc[:,'Unit'] + ' per year')
df_prod.to_csv('production_norm_filtered.csv', index=False)

/var/folders/r1/74s8bhy11bz94lsfxrg5752m0000gn/T/ipykernel_87901/3238290160.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_prod =pd.read_csv("Data/Production/Production_Crops_Livestock_E_All_Data_(Normalized).csv")


(4124611, 14)


# Food and Security Data

## Data set:

* https://www.fao.org/faostat/en/#data/FS

## Tasks performed:

* Filter to keep only data from the last surveys,
* Remove values expressed in millions when the same valmue is available as a percentage,
* filter non-essential columns,
* Formulate a sentence based on the remaining columns: 
```
f'In {Area} the {Item} is {Value} {Unit}.'
```

In [ ]:

fs_norm_df = pd.read_csv('Data/FS/Food_Security_Data_E_All_Data_(Normalized)/Food_Security_Data_E_All_Data_(Normalized).csv',encoding='latin-1')

def group_function(x):
    return pd.Series({'Max Year Code': np.max(x['Year Code'])})

group_features = ['Area Code', 'Item Code','Element Code']

grouped_df = fs_norm_df[group_features+['Year Code']].groupby(by = group_features).apply(group_function).reset_index()

fs_norm_df = fs_norm_df.merge(grouped_df, on = group_features, how= 'outer')
fs_norm_df_filtered = fs_norm_df[fs_norm_df['Year Code']==fs_norm_df['Max Year Code']]

excluded_items = [ 'severely food insecure female adults (million) (3-year average)',
 'severely food insecure female adults (million) (annual value)',
 'severely food insecure male adults (million) (3-year average)',
 'severely food insecure male adults (million) (annual value)',
 'severely food insecure people (million) (3-year average)',
 'severely food insecure people (million) (annual value)','Number of children under 5 years affected by wasting (million)',
 'Number of children under 5 years of age who are overweight (modeled estimates) (million)',
 'Number of children under 5 years of age who are stunted (modeled estimates) (million)',
  'Number of severely food insecure female adults (million) (3-year average)',
 'Number of severely food insecure female adults (million) (annual value)',
 'Number of severely food insecure male adults (million) (3-year average)',
 'Number of severely food insecure male adults (million) (annual value)',
 'Number of severely food insecure people (million) (3-year average)',
 'Number of severely food insecure people (million) (annual value)',
  'Number of moderately or severely food insecure female adults (million) (3-year average)',
 'Number of moderately or severely food insecure female adults (million) (annual value)',
 'Number of moderately or severely food insecure male adults (million) (3-year average)',
 'Number of moderately or severely food insecure male adults (million) (annual value)',
 'Number of moderately or severely food insecure people (million) (3-year average)',
 'Number of moderately or severely food insecure people (million) (annual value)',
 'Number of newborns with low birthweight (million)',
 'Number of people undernourished (million) (3-year average)',
 'Prevalence of moderate or severe food insecurity in the rural adult population (percent) (annual value)',
 'Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average)',
 'Prevalence of moderate or severe food insecurity in the total population (percent) (annual value)',
 'Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average)',
 'Prevalence of moderate or severe food insecurity in the total population (percent) (annual value)',
 'Prevalence of moderate or severe food insecurity in the town and semi-dense area adult population (percent) (annual value)',
 'Prevalence of moderate or severe food insecurity in the urban adult population (percent) (annual value)',
  'Prevalence of severe food insecurity in the rural adult population (percent) (annual value)',
 'Prevalence of severe food insecurity in the total population (percent) (3-year average)',
 'Prevalence of severe food insecurity in the total population (percent) (annual value)',
 'Number of women of reproductive age (15-49 years) affected by anemia (million)',
  'Prevalence of severe food insecurity in the total population (percent) (3-year average)',
 'Prevalence of severe food insecurity in the total population (percent) (annual value)',
 'Prevalence of severe food insecurity in the town and semi-dense area adult population (percent) (annual value)',
 'Prevalence of severe food insecurity in the urban adult population (percent) (annual value)',]
fs_norm_df_filtered = fs_norm_df_filtered[~fs_norm_df_filtered.Item.isin(excluded_items)]

columns_to_keep = ['Area', 'Item' , 'Unit', 'Value']
fs_norm_df_filtered = fs_norm_df_filtered[columns_to_keep]

fs_norm_df_filtered.loc[:,'Sentence'] = ('In ' + fs_norm_df_filtered.loc[:,'Area'] 
                                         + ' the ' + fs_norm_df_filtered.loc[:,'Item'] 
                                         + ' is ' + fs_norm_df_filtered.loc[:,'Value'].astype(str) 
                                         + ' ' + fs_norm_df_filtered.loc[:,'Unit'])

fs_norm_df_filtered.to_csv('fs_norm_filtered.csv')

/var/folders/r1/74s8bhy11bz94lsfxrg5752m0000gn/T/ipykernel_16470/817154024.py:1: DtypeWarning: Columns (16,22,40,46,52,58,70,82,88,94,97,100,103,109,115,118,121,124,127,130,133,136,139,143,145) have mixed types. Specify dtype option on import or set low_memory=False.
  fs_df = pd.read_csv('Data/FS/Food_Security_Data_E_All_Data/Food_Security_Data_E_All_Data.csv',encoding='latin-1')
